In [9]:
import torch
import pandas as pd

### DE-En module

In [15]:
# load pre-trained En→De translation module

en2de = torch.hub.load('pytorch/fairseq', 'transformer.wmt19.en-de.single_model')
en2de.cuda()

Using cache found in /home/docando/.cache/torch/hub/pytorch_fairseq_master


GeneratorHubInterface(
  (models): ModuleList(
    (0): TransformerModel(
      (encoder): TransformerEncoder(
        (dropout_module): FairseqDropout()
        (embed_tokens): Embedding(42024, 1024, padding_idx=1)
        (embed_positions): SinusoidalPositionalEmbedding()
        (layers): ModuleList(
          (0): TransformerEncoderLayer(
            (self_attn): MultiheadAttention(
              (dropout_module): FairseqDropout()
              (k_proj): Linear(in_features=1024, out_features=1024, bias=True)
              (v_proj): Linear(in_features=1024, out_features=1024, bias=True)
              (q_proj): Linear(in_features=1024, out_features=1024, bias=True)
              (out_proj): Linear(in_features=1024, out_features=1024, bias=True)
            )
            (self_attn_layer_norm): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
            (dropout_module): FairseqDropout()
            (activation_dropout_module): FairseqDropout()
            (fc1): Linear(in_feat

In [17]:
# 동작 test
en2de.translate('Hello world', beam=5)

'Hallo Welt'

### En-De module

In [20]:
# load pre-trained De→En translation module

de2en = torch.hub.load('pytorch/fairseq', 'transformer.wmt19.de-en.single_model')
de2en.cuda()

Using cache found in /home/docando/.cache/torch/hub/pytorch_fairseq_master


GeneratorHubInterface(
  (models): ModuleList(
    (0): TransformerModel(
      (encoder): TransformerEncoder(
        (dropout_module): FairseqDropout()
        (embed_tokens): Embedding(42024, 1024, padding_idx=1)
        (embed_positions): SinusoidalPositionalEmbedding()
        (layers): ModuleList(
          (0): TransformerEncoderLayer(
            (self_attn): MultiheadAttention(
              (dropout_module): FairseqDropout()
              (k_proj): Linear(in_features=1024, out_features=1024, bias=True)
              (v_proj): Linear(in_features=1024, out_features=1024, bias=True)
              (q_proj): Linear(in_features=1024, out_features=1024, bias=True)
              (out_proj): Linear(in_features=1024, out_features=1024, bias=True)
            )
            (self_attn_layer_norm): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
            (dropout_module): FairseqDropout()
            (activation_dropout_module): FairseqDropout()
            (fc1): Linear(in_feat

In [21]:
# 동작 test
de2en.translate('Hallo Welt', beam=5)

'Hello World'

### Data Load

In [22]:
def load_data():
    train = pd.read_csv("../data/train_final.csv")
    eval = pd.read_csv("../data/eval_final_open.csv")
    train_X = train['Sentence'] # 본문 내용을 기준으로 분류하기
    train_Y = train['Category']
    eval_X = eval['Sentence']
    return train_X, train_Y, eval_X

In [23]:
train_x, train_y, test_x = load_data()

In [24]:
train_x

0        -LRB- The film -RRB- tackles the topic of rela...
1                    Lavishly , exhilaratingly tasteless .
2                           It is also beautifully acted .
3        But , like Silence , it 's a movie that gets u...
4        It 's been made with an innocent yet fervid co...
                               ...                        
11539    Although Frailty fits into a classic genre , i...
11540                   Mediocre fable from Burkina Faso .
11541    Like all great films about a life you never kn...
11542    Those who are n't put off by the film 's auste...
11543    An ambitious movie that , like Shiner 's organ...
Name: Sentence, Length: 11544, dtype: object

In [25]:
en2de_train_x = en2de.translate(train_x, beam=5)

In [26]:
en2de_train_x

['-LRB- Der Film -RRB- behandelt das Thema Beziehungen auf eine so direkte, emotional ehrliche Weise, dass am Ende unmöglich festzustellen ist, ob der Film im Kern zutiefst pessimistisch oder still hoffnungsvoll ist.',
 'Verschwenderisch, berauschend geschmacklos.',
 'Es ist auch wunderschön gespielt.',
 'Aber wie Schweigen ist es ein Film, der unter die Haut geht.',
 'Es wurde mit der unschuldigen, aber leidenschaftlichen Überzeugung gemacht, dass unser Hollywood so gut wie verloren hat.',
 'Regisseur David Jacobson gibt Dahmer zu bedenken, dass der Mörder seine Opfer nie verspielt.',
 'Insgesamt ist Cletis Tout eine preisgekrönte Komödie, die die Fantasie anregt und den komischen Knochen kitzelt.',
 'Der Film ist zu sehr von seinen eigenen ernsten Einsichten beeindruckt, als dass er viel Unterhaltungswert aufarbeiten könnte.',
 '<unk> -LRB- Hopkins -RRB- telefoniert in seiner Performance weniger als faxt sie.',
 'Herrlicher Spaß.',
 'Zu fad und geschmacklos, um wirklich prüde zu sein

In [27]:
de2en_train_x = de2en.translate(en2de_train_x, beam=5)

In [28]:
de2en_train_x

['-LRB- The film -RRB- deals with the subject of relationships in such a direct, emotionally honest way that in the end it is impossible to determine whether the film is profoundly pessimistic or quietly hopeful at its core.',
 'Wasteful, intoxicatingly tasteless.',
 "It's also beautifully played.",
 "But like silence, it's a film that gets under your skin.",
 'It was done with an innocent but passionate conviction that our Hollywood has all but lost.',
 'Director David Jacobson tells Dahmer that the killer never gambles away his victims.',
 'Overall, Cletis Tout is an award-winning comedy that captures the imagination and tickles the comic bone.',
 'The film is too impressed by its own serious insights to absorb much entertainment value.',
 '< br > -LRB- Hopkins -RRB- is less than faxed in his performance.',
 'Gorgeous fun.',
 'Too bland and distasteful to be truly prudish.',
 "Ozpetek's efforts have the scale and form of a particularly successful television film.",
 'When the Mediter

In [29]:
# data저장

pd.DataFrame(de2en_train_x).to_csv('../data/train_x_de2en.csv', header=

In [32]:
train_y_li = train_y.to_list()

In [ ]:
data_dict = {'Category': train_y_li, 'Sentence': de2en_train_x}
dfpd.DataFrame(data_dict)